# Dataset

In [ ]:
from keras.datasets import mnist
(X, y), (X_test, y_test) = mnist.load_data(path="mnist.npz")

In [ ]:
X.shape, y.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [ ]:
X, X_test = X/255, X_test/255

In [ ]:
X_train, y_train = X[:50000], y[:50000]
X_val, y_val = X[50000:], y[50000:]

# Model

In [ ]:
from keras import Model, Input
from keras.layers import Dense, Flatten, Reshape
from keras import activations, losses, metrics
from keras.optimizers import Adam

In [ ]:
# Encodeur
encodeur_input = Input(shape = (28, 28))
x0 = Flatten()(encodeur_input)
x1 = Dense(128, activation = activations.relu)(x0)
x2 = Dense(8, activation = activations.relu)(x1)
mu = Dense(2, activation=activations.relu)(x2)
sigma = Dense(2, activation=activations.relu)(x2)
z = mu + sigma
encodeur = Model(inputs = encodeur_input, outputs = [z, mu, sigma])

# Decodeur
decodeur_inputs = Input(shape = (2, ))
x3 = Dense(8, activation = activations.relu)(decodeur_inputs)
x4 = Dense(128, activation = activations.relu)(x3)
x5 = Dense(784, activation = activations.sigmoid)(x4)
decodeur_outputs = Reshape(target_shape=(28, 28))(x5)
decodeur = Model(inputs = decodeur_inputs, outputs = decodeur_outputs)

# Autoencodeur
autoencodeur_variationnel = Model(inputs = encodeur_input, outputs = [decodeur(encodeur(encodeur_input)), mu, sigma])

decodeur = Model(decodeur_inputs, x5)